In [1]:
"""
   Copyright 2019-2021 Boris Shminke

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
"""
!pip install neural-semigroups

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
from neural_semigroups import Magma
from neural_semigroups.utils import hide_cells, partial_table_to_cube
import torch
from typing import Tuple

# examples for other cardinalities are located in recspective directories
cardinality = 4


def transform(
        cayley_table: Tuple[torch.Tensor, ...]
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    this is a data augmentation function similar to those which are used for
    pictures, but insted of rotations and translations we use other
    transformations

    :param cayley_table: a tuple from a single tensor --- a Cayley table of
        some magma
    :returns: a tuple of a probabilistic representations of a partial Cayley
        table and the respecting full one
    """
    # in the ``smallsemi`` package they store only table up to isomorphism _or_
    # anti-isomorphism, that's why here we apply isomorphisms or
    # anti-isomorphisms with equal probabilies
    if torch.randn((1, )).cpu().item() > 0.5:
        full_table = Magma(cayley_table[0]).random_isomorphism()
    else:
        full_table = Magma(cayley_table[0]).random_isomorphism().T
    # we want to reconstruct an math:``n\times n`` table given math:``n`` cells
    partial_cube = partial_table_to_cube(
        hide_cells(
            full_table,
            cardinality * cardinality - cardinality
        )
    )
    return partial_cube,  partial_table_to_cube(full_table)

In [3]:
from neural_semigroups.smallsemi_dataset import Smallsemi

# when running not on Kaggle, one can set ``download`` parameter to ``True`` to
# download original ``smallsemi`` databases
data = Smallsemi(
    root="/kaggle/input/smallsemi",
    cardinality=cardinality,
    transform=transform
)

In [4]:
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader

data_size = len(data)
print(data_size)
# depending on the size of available data we leave either 1024 equivalence
# classes or one third of them (whichever is smaller) for each training and
# validation sets
test_size = min(len(data) // 3, 1024)
data_loaders = tuple(
    DataLoader(data_split, batch_size=32)
    for data_split
    in random_split(data, [data_size - 2 * test_size, test_size, test_size])
)

126


Since we try to reconstruct an associative Cayley table from only a handfull of cells, it's hardly probable that we end up with an original table. Most of the time the network returns other solutions for the same task (since the solution is none unique). That's why we used a special loss function described [here](https://neural-semigroups.readthedocs.io/en/latest/package-documentation.html#associator-loss).

In [5]:
from neural_semigroups.associator_loss import AssociatorLoss
from torch import Tensor

def loss(prediction: Tensor, target: Tensor) -> Tensor:
    return AssociatorLoss()(prediction)

In [6]:
from neural_semigroups import MagmaDAE

# here we use a typical hourglass shape of an autoencoder
dae = MagmaDAE(
    cardinality=cardinality,
    hidden_dims=[cardinality ** 3, cardinality ** 2, cardinality]
)

Since we want to upload training logs to tensorboard.dev, we first remove all the previous experiments from the same folder

In [7]:
!rm -rf runs

In [8]:
from neural_semigroups.training_helpers import learning_pipeline
from ignite.metrics.loss import Loss
from neural_semigroups.training_helpers import associative_ratio, guessed_ratio

params = {"learning_rate": 0.001, "epochs": 1000}
metrics = {
    "loss": Loss(loss),
    "associative_ratio": Loss(associative_ratio),
    "guessed_ratio": Loss(guessed_ratio)
}
learning_pipeline(params, dae, loss, metrics, data_loaders)
torch.onnx.export(
    dae,
    next(iter(data_loaders[0]))[0],
    f"dae-{cardinality}.onnx"
)

The experimental results can be found [here](https://tensorboard.dev/experiment/k8LaiW1oTZef8d9oo8J96w).